In [3]:
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.keras.__version__)

2.4.1
2.4.0


In [4]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [5]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_crossentropy])

In [6]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))

model.fit(train_x, train_y, epochs=10, batch_size=100, validation_data=(val_x, val_y))

Epoch 1/10
10/10 [==============================] - 1s 68ms/step - loss: 11.8163 - categorical_crossentropy: 11.8163 - val_loss: 11.9598 - val_categorical_crossentropy: 11.9598
Epoch 2/10
10/10 [==============================] - 0s 5ms/step - loss: 12.0057 - categorical_crossentropy: 12.0057 - val_loss: 12.5072 - val_categorical_crossentropy: 12.5072
Epoch 3/10
10/10 [==============================] - 0s 5ms/step - loss: 12.5956 - categorical_crossentropy: 12.5956 - val_loss: 13.7200 - val_categorical_crossentropy: 13.7200
Epoch 4/10
10/10 [==============================] - 0s 6ms/step - loss: 14.1626 - categorical_crossentropy: 14.1626 - val_loss: 15.9017 - val_categorical_crossentropy: 15.9017
Epoch 5/10
10/10 [==============================] - 0s 6ms/step - loss: 16.7608 - categorical_crossentropy: 16.7608 - val_loss: 18.9345 - val_categorica

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.batch(32)
dataset = dataset.repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(32)
val_dataset = val_dataset.repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30, validation_data=val_dataset, validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 7ms/step - loss: 57.3681 - categorical_crossentropy: 57.3681 - val_loss: 69.4665 - val_categorical_crossentropy: 69.4665
Epoch 2/10
30/30 [==============================] - 0s 3ms/step - loss: 89.8610 - categorical_crossentropy: 89.8610 - val_loss: 103.9787 - val_categorical_crossentropy: 103.9787
Epoch 3/10
30/30 [==============================] - 0s 2ms/step - loss: 129.6414 - categorical_crossentropy: 129.6414 - val_loss: 146.9704 - val_categorical_crossentropy: 146.9704
Epoch 4/10
30/30 [==============================] - 0s 2ms/step - loss: 180.1554 - categorical_crossentropy: 180.1554 - val_loss: 200.2292 - val_categorical_crossentropy: 200.2292
Epoch 5/10
30/30 [==============================] - 0s 3ms/step - loss: 240.0888 - categorical_crossentropy: 240.0888 - val_loss: 261.3928 - val

In [8]:
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=32)

32/32 [==============================] - 0s 1ms/step - loss: 581.1237 - categorical_crossentropy: 581.1237


[581.1236572265625, 581.1236572265625]

In [9]:
result = model.predict(test_x, batch_size=32)
print(result.shape)

(1000, 10)


### 函数式API

In [10]:
input_x = tf.keras.Input(shape=(72, ))
hidden1 = layers.Dense(32, activation='relu')(input_x)
hidden2 = layers.Dense(16, activation='relu')(hidden1)
pred = layers.Dense(10, activation='softmax')(hidden2)

model = tf.keras.Model(inputs=input_x, outputs=pred)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
model.fit(train_x, train_y, epochs=10, batch_size=32)

Epoch 1/10
32/32 [==============================] - 0s 1ms/step - loss: 12.6747 - accuracy: 0.0794
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 19.4658 - accuracy: 0.1008
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 35.5492 - accuracy: 0.0897
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 67.4838 - accuracy: 0.0847
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 117.0065 - accuracy: 0.0991
Epoch 6/10
32/32 [======

### 模型子类化

通过tf.keras.Model

* 在_init_ 方法中创建层并将它们设置为类实例的属性。
* 在_call_方法中定义前向传播

In [11]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.layer1 = layers.Dense(32, activation='relu')
        self.layer2 = layers.Dense(self.num_classes, activation='softmax')

    def call(self, inputs):
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

model = MyModel(num_classes=10)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
model.fit(train_x, train_y, epochs=5, batch_size=16)

Epoch 1/5
63/63 [==============================] - 1s 2ms/step - loss: 12.8768 - accuracy: 0.0972
Epoch 2/5
63/63 [==============================] - 0s 2ms/step - loss: 22.0442 - accuracy: 0.1176
Epoch 3/5
63/63 [==============================] - 0s 1ms/step - loss: 34.1015 - accuracy: 0.1094
Epoch 4/5
63/63 [==============================] - 0s 917us/step - loss: 42.9357 - accuracy: 0.1140

###　自定义层

通过tf.keras.layers.Layer

* _init_: (可选)定义该层要使用的子层
* build：创建层的权重。使用 add_weight 方法添加权重。
* call：定义前向传播。
* compute_output_shape：指定在给定输入形状的情况下如何计算层的输出形状。
* 可选，可以通过实现 get_config 方法和 from_config 类方法序列化层。

In [12]:
class MyLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.output_dim = output_dim

    def build(self, input_shape):
        print("input_shape: %s" % input_shape)
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape,
                                      initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

model.fit(train_x, train_y, epochs=5, batch_size=32)
print(model.summary())

Epoch 1/5
input_shape: (None, 72)
32/32 [==============================] - 1s 901us/step - loss: 11.6155 - accuracy: 0.0902
Epoch 2/5
32/32 [==============================] - 0s 901us/step - loss: 11.6586 - accuracy: 0.0936
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 11.7489 - accuracy: 0.1051
Epoch 4/5
32/32 [==============================] - 0s 997us/step - loss: 11.6767 - accuracy: 0.1071
Epoch 5/5
32/32 [==============================] - 0s 772us/step - loss: 11.7377 - accuracy: 0.0887
Model: "sequential_1"
___________________________________________________________

### 回调
回调是传递给模型以自定义和扩展其在训练期间的行为的对象。我们可以编写自己的自定义回调，或使用tf.keras.callbacks中的内置函数，常用内置回调函数如下：

* tf.keras.callbacks.ModelCheckpoint：定期保存模型的检查点。
* tf.keras.callbacks.LearningRateScheduler：动态更改学习率。
* tf.keras.callbacks.EarlyStopping：验证性能停止提高时进行中断培训。
* tf.keras.callbacks.TensorBoard：使用TensorBoard监视模型的行为 。

In [13]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir="./logs")
]
model.fit(train_x, train_y, batch_size=16, epochs=5,
          callbacks=callbacks, validation_data=(val_x, val_y))

Epoch 1/5
63/63 [==============================] - 1s 9ms/step - loss: 11.6655 - accuracy: 0.0970 - val_loss: 11.7177 - val_accuracy: 0.1050
Epoch 2/5
63/63 [==============================] - 0s 2ms/step - loss: 11.6680 - accuracy: 0.1020 - val_loss: 11.7201 - val_accuracy: 0.0950
Epoch 3/5
63/63 [==============================] - 0s 2ms/step - loss: 11.6694 - accuracy: 0.0890 - val_loss: 11.7175 - val_accuracy: 0.1100
Epoch 4/5
63/63 [==============================] - 0s 

### 模型保存和恢复

#### 权重保存

In [14]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32, )), # 需要input_shape，不然save_weights报错
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [15]:
model.save_weights('./weights/model')
model.load_weights('./weights/model')
model.save_weights('./model.h5', save_format='h5')
model.load_weights('./model.h5')

#### 保存网络结构

In [16]:
import json
import pprint

json_str = model.to_json()
pprint.pprint(json.loads(json_str))
fresh_model = tf.keras.models.model_from_json(json_str)


{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'InputLayer',
                        'config': {'batch_input_shape': [None, 32],
                                   'dtype': 'float32',
                                   'name': 'dense_8_input',
                                   'ragged': False,
                                   'sparse': False}},
                       {'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'batch_input_shape': [None, 32],
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {}},
                                   'bias_regularizer': None,
                                   'dtype': 'float32',
                                   'kerne

In [18]:
yaml_str = model.to_yaml()
print(yaml_str)
fresh_model = tf.keras.models.model_from_yaml(yaml_str)

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: InputLayer
    config:
      batch_input_shape: !!python/tuple
      - null
      - 32
      dtype: float32
      name: dense_8_input
      ragged: false
      sparse: false
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple
      - null
      - 32
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config:
          seed: null
      kernel_regularizer: null
      name: dense_8
      trainable: true
      units: 64
      use_bias: true
  - class_name: Dense
    config:
      activation: softmax
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularize

#### 保存整个模型

In [21]:
model = tf.keras.Sequential([
    layers.Dense(10, activation='softmax', input_shape=(72, )),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)
model.save('all_model.h5')
model = tf.keras.models.load_model('all_model.h5')


Epoch 1/5
32/32 [==============================] - 0s 836us/step - loss: 11.6601 - accuracy: 0.0874
Epoch 2/5
32/32 [==============================] - 0s 965us/step - loss: 11.6785 - accuracy: 0.0691
Epoch 3/5
32/32 [==============================] - 0s 997us/step - loss: 11.7556 - accuracy: 0.0972
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 11.7213 - accuracy: 0.1396
Epoch 5/5
32/32 [==============================] - 0s 804us/step - loss: 11.7837 - accuracy: 0.1238


### 将keras用于Estimator

Estimator API 用于针对分布式环境训练模型。它适用于一些行业使用场景，例如用大型数据集进行分布式训练并导出模型以用于生产

In [23]:
model = tf.keras.Sequential([layers.Dense(10,activation='softmax'),
                          layers.Dense(10,activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

estimator = tf.keras.estimator.model_to_estimator(model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\LINYUR~1\\AppData\\Local\\Temp\\tmp0qm8vq3g', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas'

In [24]:

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = tf.keras.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
    model.add(layers.Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.SGD(0.2)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 16)                176       
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [25]:
x = np.random.random((1024, 10))
y = np.random.randint(2, size=(1024, 1))
x = tf.cast(x, tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=1024).batch(32)
model.fit(dataset, epochs=1)

32/32 [==============================] - 3s 1ms/step - loss: 0.7031
